# Construire un EAF à partir d'un fichier de sous-titres issu de Whisper

In [129]:
import lxml.etree as ET
import codecs

## Construction de la coquille
- racine ANNOTATION_DOCUMENT
        

In [130]:
ANNOTATION_DOCUMENT=ET.Element("ANNOTATION_DOCUMENT",AUTHOR="",DATE="",FORMAT="3.0", VERSION="3.0")

### Header
- header
     - media_descriptor
     - property
 

In [131]:
HEADER=ET.SubElement(ANNOTATION_DOCUMENT,"HEADER",MEDIA_FILE="",TIME_UNITS="milliseconds")
MEDIA_DESCRIPTOR=ET.SubElement(HEADER,"MEDIA_DESCRIPTOR",MEDIA_URL="",MIME_TYPE="",RELATIVE_MEDIA_URL="")
PROPERTY=ET.SubElement(HEADER,"PROPERTY", NAME="lastUsedAnnotationId")

### Time order for Time slots
- time order
    - time slot


In [132]:
TIME_ORDER=ET.SubElement(ANNOTATION_DOCUMENT,"TIME_ORDER")

### Tiers for Annotations
- tier
    - annotation
        - alignable annotation
            - annotation value


In [133]:
WhisperTIER=ET.SubElement(ANNOTATION_DOCUMENT,"TIER",ANNOTATOR="Whisper",LINGUISTIC_TYPE_REF="defaut-lt",PARTICIPANT="All",TIER_ID="Whisper")

In [134]:
JournalTIER=ET.SubElement(ANNOTATION_DOCUMENT,"TIER",ANNOTATOR="",LINGUISTIC_TYPE_REF="defaut-lt",PARTICIPANT="Journaliste",TIER_ID="Journaliste")

In [135]:
ExpertTIER=ET.SubElement(ANNOTATION_DOCUMENT,"TIER",ANNOTATOR="",LINGUISTIC_TYPE_REF="defaut-lt",PARTICIPANT="Expert",TIER_ID="Expert")

In [136]:
TerrainTIER=ET.SubElement(ANNOTATION_DOCUMENT,"TIER",ANNOTATOR="",LINGUISTIC_TYPE_REF="defaut-lt",PARTICIPANT="Terrain",TIER_ID="Terrain")

### Linguistic type

In [137]:
LINGUISTIC_TYPE=ET.SubElement(ANNOTATION_DOCUMENT,"LINGUISTIC_TYPE",GRAPHIC_REFERENCES="false",LINGUISTIC_TYPE_ID="defaut-lt",TIME_ALIGNABLE="true")

## Conversion des sous-titres vers EAF

In [138]:
with codecs.open("FranceArgentine-DecWhisper-TransGold.srt","r",encoding="utf8") as inFile:
    lignesSRT=[l.strip() for l in inFile.readlines()]
if lignesSRT[0].endswith("1"): lignesSRT[0]="1"
else: print("vérifier le fichier SRT")

In [139]:
def getSeconds(time):
    h, m, s = [float(i) for i in time.replace(",",".").split(':')]
    result=((60*h+m)*60+s)*1000
    return str(int(result))

def getSubtitles(lignes):
    result=[]
    currentState="start"
    currentNum=1
    for l in lignes:
        if currentState=="start" and str(currentNum)==l:
            num=l
            currentState="time"
        elif currentState=="time":
            t1,t2=l.split(" --> ")
            t1=getSeconds(t1)
            t2=getSeconds(t2)
            currentState="text"
        elif currentState=="text":
            text=l
            currentState="skip"
        elif currentState=="skip" and l=="":
            currentState="start"
            currentNum+=1
            result.append({"num":num,"start":t1,"end":t2,"text":text})
    return result        

In [140]:
subtitles=getSubtitles(lignesSRT)

In [141]:
for s in subtitles:
    ANNOTATION=ET.SubElement(WhisperTIER,"ANNOTATION")
    aid="au"+s["num"]
    ALIGNABLE_ANNOTATION=ET.SubElement(ANNOTATION,"ALIGNABLE_ANNOTATION",
                                       ANNOTATION_ID=aid,
                                       TIME_SLOT_REF1=aid+s["start"],
                                       TIME_SLOT_REF2=aid+s["end"]
                                      )
    ANNOTATION_VALUE=ET.SubElement(ALIGNABLE_ANNOTATION,"ANNOTATION_VALUE")
    ANNOTATION_VALUE.text=s["text"]
    TIME_SLOT=ET.SubElement(TIME_ORDER,"TIME_SLOT",
                            TIME_SLOT_ID=aid+s["start"],
                            TIME_VALUE=s["start"],
                           )
    TIME_SLOT=ET.SubElement(TIME_ORDER,"TIME_SLOT",
                            TIME_SLOT_ID=aid+s["end"],
                            TIME_VALUE=s["end"],
                           )

## Exportation du fichier EAF

In [142]:
print (ET.tostring(ANNOTATION_DOCUMENT,pretty_print=True,encoding="utf8").decode("utf8"))

<ANNOTATION_DOCUMENT AUTHOR="" DATE="" FORMAT="3.0" VERSION="3.0">
  <HEADER MEDIA_FILE="" TIME_UNITS="milliseconds">
    <MEDIA_DESCRIPTOR MEDIA_URL="" MIME_TYPE="" RELATIVE_MEDIA_URL=""/>
    <PROPERTY NAME="lastUsedAnnotationId"/>
  </HEADER>
  <TIME_ORDER>
    <TIME_SLOT TIME_SLOT_ID="au129560" TIME_VALUE="29560"/>
    <TIME_SLOT TIME_SLOT_ID="au136560" TIME_VALUE="36560"/>
    <TIME_SLOT TIME_SLOT_ID="au236560" TIME_VALUE="36560"/>
    <TIME_SLOT TIME_SLOT_ID="au239320" TIME_VALUE="39320"/>
    <TIME_SLOT TIME_SLOT_ID="au339320" TIME_VALUE="39320"/>
    <TIME_SLOT TIME_SLOT_ID="au351480" TIME_VALUE="51480"/>
    <TIME_SLOT TIME_SLOT_ID="au451480" TIME_VALUE="51480"/>
    <TIME_SLOT TIME_SLOT_ID="au456560" TIME_VALUE="56560"/>
    <TIME_SLOT TIME_SLOT_ID="au556560" TIME_VALUE="56560"/>
    <TIME_SLOT TIME_SLOT_ID="au560920" TIME_VALUE="60920"/>
    <TIME_SLOT TIME_SLOT_ID="au660920" TIME_VALUE="60920"/>
    <TIME_SLOT TIME_SLOT_ID="au667920" TIME_VALUE="67920"/>
    <TIME_SLOT TIME

In [143]:
EAF=ET.ElementTree(ANNOTATION_DOCUMENT)
EAF.write("test.eaf", pretty_print=True, encoding='utf-8', xml_declaration=True)